# Computational Intelligence in Chemistry - Hands-on Active Learning
### Luis Filipe Menezes - RA: 164924
This is part of an assignment for the Computational Intelligence in Chemistry course on UNIFESP's masters program.

This notebook must:

I. Use the full QM9 dataset.

II. Select one representation for the molecules (EV-CM,
MBTR, Mordred, etc.)

III. Setup a topline predictor:
  - I.Select a method, e.g., MLP, RF, KRR, etc.,
  - II.Split the dataset into train (80%), validation (10%) and test (10%)
  - III.Perform the model hyperparameter tuning

IV. Evaluate the final model with the test set

IV. Active Learning: Committee-base approach
  - I. Using the baseline model established in step III: train multiple regressors (with different seed and small hyperparameters modifications)
  - II.Use a subset of 1% of the training data to build the initial models
  - III.At each step of the active learning loop, select new samples from the training data (the number of samples must be evaluated – select a small number)
  - IV.Update the model with the new samples

V. Plot a graph showing the Error measurement versus the percentage of samples used for training

VI. Compare the active learning results with a passive (random) selection

## I. Loading the full QM9 dataset

In [1]:
!pip install --quiet dscribe

In [2]:
!wget -O data.tar.bz2 https://www.dropbox.com/scl/fi/2ugqxr9fa9nob1byc8ura/dsgdb9nsd.xyz.tar.bz2?rlkey=pp2k6fy4360yldrypwghwbi6d&st=1cohswqh&dl=0
!!mkdir qm9_files
!tar -xjf data.tar.bz2 -C qm9_files/

--2025-11-17 13:02:56--  https://www.dropbox.com/scl/fi/2ugqxr9fa9nob1byc8ura/dsgdb9nsd.xyz.tar.bz2?rlkey=pp2k6fy4360yldrypwghwbi6d
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98921f81139f6c0c48b1b8c49b.dl.dropboxusercontent.com/cd/0/inline/C1UpepAhjM_8nC3PIWnBWldVs5TGGpZmH4R7UrMPKjV7P-r-bUi-nN74F8n5SN8d8hU0k_Oa9tT1EwemLepckMEc9yDfxdQ0nGO27gR0KAfE94uMIav5YDhMILsLF8n7sG5miedoRFcWtNNui7WURgVh/file# [following]
--2025-11-17 13:02:57--  https://uc98921f81139f6c0c48b1b8c49b.dl.dropboxusercontent.com/cd/0/inline/C1UpepAhjM_8nC3PIWnBWldVs5TGGpZmH4R7UrMPKjV7P-r-bUi-nN74F8n5SN8d8hU0k_Oa9tT1EwemLepckMEc9yDfxdQ0nGO27gR0KAfE94uMIav5YDhMILsLF8n7sG5miedoRFcWtNNui7WURgVh/file
Resolving uc98921f81139f6c0c48b1b8c49b.dl.dropboxusercontent.com (uc98921f81139f6c0c48b1b8c49b.dl.dropboxusercontent.com)... 162.125.5.1

In [3]:
qm9_folder = 'qm9_files/'
QM9_PROPERTIES = [
    "A", 'B', 'C', 'mu', 'alpha',
    'homo', 'lumo', 'gap', 'r2',
    'zpve', 'U0', 'U', 'H', 'G', 'Cv'
]


with open(qm9_folder+'dsgdb9nsd_113885.xyz') as file:
  for line in file:
    print(line)

19

gdb 113885	3.70802	0.80999	0.7081	2.6975	73.65	-0.2639	0.0141	0.278	1593.5236	0.156316	-460.174889	-460.164623	-460.163679	-460.212278	35.326	

C	-0.008898706	 1.435343511	-0.1229342875	-0.480889

C	-0.097211668	-0.0611740136	 0.0533750899	 0.431334

O	-0.522805097	-0.6329577977	 1.0223608396	-0.34442

O	 0.3874153201	-0.692641184	-1.0454419183	-0.247509

C	 0.3502508225	-2.1099666389	-1.0216443391	 0.118249

C	-0.6452778548	-2.8049992501	-1.9010293635	-0.305338

C	 0.8299955957	-2.806393386	-2.257736521	-0.04378

C	 1.3090797685	-2.0281448361	-3.4552388748	-0.093611

O	 1.0953742585	-2.8320000843	-4.6085691448	-0.419869

H	-0.3728027153	 1.9278328937	 0.7777832851	 0.149542

H	 1.0248947898	 1.7315350968	-0.3210999853	 0.149596

H	-0.6070917791	 1.7459702376	-0.9844780335	 0.149165

H	 0.576972916	-2.5190329042	-0.0453019279	 0.12476

H	-1.1102986284	-3.7153217605	-1.5406769745	 0.122235

H	-1.2860640388	-2.1730656326	-2.5065209428	 0.123942

H	 1.3703384329	-3.7344941519	-2.09474

In [4]:
import os
import pandas as pd
import numpy as np
from ase import Atoms


np.random.seed(24)
N_SUBSET_SAMPLES = 5000

def load_validate_mol(file_string):
  with open(file_string, 'r') as file:
    n_atoms = int(file.readline())

    properties_list = file.readline().split()
    mol_id = properties_list[1]
    properties_values = [float(prop) for prop in properties_list[2:]]
    properties_dict = dict(zip(QM9_PROPERTIES, properties_values))


    atoms = []
    atoms_coords = []
    for nlines in range(n_atoms):
      atom_info = file.readline().split()
      atoms.append(atom_info[0]) # get only the the atom string

      atom_coords_str = atom_info[1:4]
      # print(atom_coords)
      try:
        atom_coords = [float(coord.replace('*^', 'e')) for coord in atom_coords_str]
        atoms_coords.append(atom_coords)

      except Exception as e:
        print(f"Couldn't extract xyz from file {file_string}: ", e)
      # print(atom_info)
    # print(atoms)
    file.readline() # ignores the frequencies

    # smiles_list = file.readline().split()


    # mol = Chem.MolFromSmiles(smiles_list[0]) # If it's None then smiles is invalid
    # if mol is None:
    #     print("Invalid smiles for arquive: ", mol_id)
    #     return
    # canon_smiles = Chem.CanonSmiles(smiles_list[0])

    mol_ase = Atoms(symbols=atoms, positions= atoms_coords)

    molecule_data = {
                'id': mol_id,
                'n_atoms': n_atoms,
                'atom_list': atoms,
                'atom_coords': atoms_coords,
                'mol_ase': mol_ase,
                # 'smiles': canon_smiles,
            }
            # Adiciona as 12 propriedades ao dicionário principal
    molecule_data.update(properties_dict)
    return molecule_data


def get_full_dataset(folder):
  dataset_list = []
  # print(f"Lendo arquivos da pasta: {folder}")

  files_to_process = [f for f in os.listdir(folder) if f.endswith('.xyz')]

  # files_to_process = np.random.choice(files_to_process, size=N_SUBSET_SAMPLES, replace=False)

  n_max_atoms = -1

  for i, file in enumerate(files_to_process):

      if (i + 1) % 10000 == 0:
          print(f"  Processando arquivo {i+1}/{len(files_to_process)}...")

      file_path = os.path.join(folder, file)

      # Chama a função de processamento para um arquivo
      molecule_data = load_validate_mol(file_path)

      n_atoms = molecule_data['n_atoms']
      if n_atoms > n_max_atoms:
          n_max_atoms = n_atoms
      # Se a função retornou dados válidos (não None), adiciona à lista
      if molecule_data:
          dataset_list.append(molecule_data)
  return dataset_list, n_max_atoms




if __name__ == '__main__':
  full_dataset, n_max_atoms = get_full_dataset(qm9_folder)

  df = pd.DataFrame(full_dataset)
  display(df.head())
  # print(load_validate_mol(qm9_folder+'dsgdb9nsd_113885.xyz')['atom_coords'])

  Processando arquivo 1000/133885...
  Processando arquivo 2000/133885...
  Processando arquivo 3000/133885...
  Processando arquivo 4000/133885...
  Processando arquivo 5000/133885...
  Processando arquivo 6000/133885...
  Processando arquivo 7000/133885...
  Processando arquivo 8000/133885...
  Processando arquivo 9000/133885...
  Processando arquivo 10000/133885...
  Processando arquivo 11000/133885...
  Processando arquivo 12000/133885...
  Processando arquivo 13000/133885...
  Processando arquivo 14000/133885...
  Processando arquivo 15000/133885...
  Processando arquivo 16000/133885...
  Processando arquivo 17000/133885...
  Processando arquivo 18000/133885...
  Processando arquivo 19000/133885...
  Processando arquivo 20000/133885...
  Processando arquivo 21000/133885...
  Processando arquivo 22000/133885...
  Processando arquivo 23000/133885...
  Processando arquivo 24000/133885...
  Processando arquivo 25000/133885...
  Processando arquivo 26000/133885...
  Processando arquivo

,id,n_atoms,atom_list,atom_coords,mol_ase,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv
0,98139,15,"[O, C, C, O, C, N, N, C, O, H, H, H, H, H, H]","[[0.1549063831, 1.3216784355, -0.0214637234], ...","(Atom('O', [np.float64(0.1549063831), np.float...",4.75304,0.83745,0.71513,0.8916,65.62,-0.2605,-0.0774,0.1831,1471.8011,0.111101,-491.118236,-491.108940,-491.107996,-491.153133,31.900
1,24959,13,"[O, C, N, C, C, O, C, C, N, H, H, H, H]","[[-1.0835e-05, -0.0029804877, -0.061518226], [...","(Atom('O', [np.float64(-1.0835e-05), np.float6...",5.24864,1.34813,1.07486,5.8102,65.68,-0.1930,0.0244,0.2174,1013.2969,0.092191,-452.800808,-452.794131,-452.793187,-452.831787,25.955
2,29455,15,"[C, C, C, O, C, C, C, C, O, H, H, H, H, H, H]","[[-0.0168566974, 1.5322352578, -0.0252432405],...","(Atom('C', [np.float64(-0.0168566974), np.floa...",3.42700,1.61003,1.10289,1.6354,72.15,-0.2054,0.0194,0.2248,1039.9727,0.114303,-420.656772,-420.649755,-420.648811,-420.688138,27.192
3,18435,16,"[C, C, C, C, C, C, N, N, H, H, H, H, H, H, H, H]","[[-0.4858454636, 1.4896049893, 0.003654386], [...","(Atom('C', [np.float64(-0.4858454636), np.floa...",2.99898,2.12744,1.63756,3.1716,68.58,-0.2531,0.0085,0.2616,834.9102,0.131217,-342.750549,-342.743253,-342.742308,-342.781867,27.489
4,93653,19,"[C, C, C, O, C, C, O, C, O, H, H, H, H, H, H, ...","[[0.0106949536, 1.5595131327, 0.0117654064], [...","(Atom('C', [np.float64(0.0106949536), np.float...",2.48869,1.47005,1.19284,1.2325,71.24,-0.2526,-0.0368,0.2158,1115.0688,0.156956,-460.188863,-460.179498,-460.178554,-460.222665,34.974


# II. Molecules representation.

We will use the Coulomb Matrix since it can represent the eletronic properties at a low computational cost. Other tests with the MBTR  descriptor for the full dataset have crashed Colab's local RAM.

In [6]:
from dscribe.descriptors import CoulombMatrix, MBTR

ev_cm = CoulombMatrix(n_atoms_max=n_max_atoms, permutation='eigenspectrum')

ev_cm_descriptors = ev_cm.create(df['mol_ase'], n_jobs=-1)

print(ev_cm_descriptors[0])

[ 2.07508720e+02  9.20513990e+01  5.87330421e+01  4.98974635e+01
  3.38430569e+01  1.89459947e+01  1.06080108e+01  6.39198513e+00
  2.57532557e+00 -8.95729913e-01 -7.66490766e-01 -4.81820598e-01
 -3.69176203e-01 -2.72214497e-01 -6.96462210e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
[ 2.07508720e+02  9.20513990e+01  5.87330421e+01  4.98974635e+01
  3.38430569e+01  1.89459947e+01  1.06080108e+01  6.39198513e+00
  2.57532557e+00 -8.95729913e-01 -7.66490766e-01 -4.81820598e-01
 -3.69176203e-01 -2.72214497e-01 -6.96462210e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


# III. Setup topline predictor

We will use an MLP approach, since we do not have the full description provided by MBTR we might need more complexes models to best suit our prediction problem.

### I. Preprocessing data

Using Stardard Scaler and dividing the dataset as $20\%$ test subset.

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

X_processed = scaler.fit_transform(ev_cm_descriptors)
y = df['gap']
# y_processed = scaler.fit_transform(df[QM9_PROPERTIES])

In [13]:
X_dev, X_test, y_dev, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=24)

### III. Hyperparameter tuning

We will use cross-validation to select the best hyperparameters configuration for the MLP model.

In [24]:
import keras
from keras.layers import Dense, Input

def create_model():
  mlp = keras.Sequential()
  mlp.add(Input(shape=(X_dev.shape[1],)))
  mlp.add(Dense(128, activation='relu'))
  mlp.add(Dense(64, activation='relu'))
  # mlp.add(Dense(32, activation='relu'))
  mlp.add(Dense(1))

  mlp.compile(optimizer='adam', loss='mae', metrics=['r2_score'])
  return mlp

In [25]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=24)

cv_scores = []
fold_no = 1

# 3. The Loop
for train_index, val_index in kf.split(X_dev, y_dev):
    print(f'Training Fold {fold_no}...')

    # Split data based on the indices provided by KFold
    X_train_fold, X_val_fold = X_dev[train_index], X_dev[val_index]
    y_train_fold, y_val_fold = y_dev.index[train_index], y_dev.index[val_index]

    # GENERATE A NEW MODEL (Resets weights)
    model = create_model()

    # Fit the model
    model.fit(
        X_train_fold, y_train_fold,
        epochs=50,          # Adjust based on convergence
        batch_size=32,
        verbose=0,          # 0 to reduce clutter, 1 to see progress
        validation_data=(X_val_fold, y_val_fold)
    )

    # Evaluate
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f'Fold {fold_no} MAE: {scores[0]:.4f}') # scores[1] is the metric (MAE)

    cv_scores.append(scores[1])
    fold_no += 1

print(f'Average CV MAE: {np.mean(cv_scores):.4f}')

Training Fold 1...
Fold 1 MAE: -0.0048
Training Fold 2...


KeyboardInterrupt: 

### IV Analisin in the test subset